# Trabalho final da disciplina Data Mining e Machine Learning II
## Prof: Marcos Guimarães
## Aluno: Walter Soares Malta

## Introdução

O objeto de nossa análise consiste em verificar a base de dados de clientes com o propósito de automatizar o processo de tomada de decisão para aprovação das linhas de crédito. Torna-se necessário criar um modelo de pontuação de crédito baseado em dados coletados de solicitantes recentes de crédito.

Utilizaremos ferramentas de modelagem preditiva, entretnato o modelo deverá ser interpretável, de forma a permitir que se explique de forma comprovada eobjetiva e se tenha subsidios para uma rejeição ao crédito.

A base de dados de Home Equity (HMEQ) contém informações de desempenho de empréstimos para 5.960 clientes. A nossa variável resposta (BAD) é uma variável binária que indica se o requerente é inadimplente ou não. Casos de clientes inadimplentes ocorreram em 1.189 casos (20%). Para cada cliente, foram registradas 12 variáveis descritas abaixo:

* BAD 1 = cliente inadimplente no empréstimo 0 = empréstimo reembolsado
* LOAN Montante do pedido de empréstimo
* MORTDUE Montante devido na hipoteca existente
* VALUE Valor da propriedade atual
* REASON DebtCon = consolidação da dívida HomeImp = melhoria da casa
* JOB Categorias profissionais JOBSix
* YOJ Anos no emprego atual
* DEROG Número de principais relatórios depreciativos
* DELINQ Número de linhas de crédito inadimplentes
* CLAGE Idade da linha comercial mais antiga em meses
* NINQ Número de linhas de crédito recentes
* CLNO Número de linhas de crédito
* DEBTINC Razão Dívida / Renda


## Desenvolvimento

In [ ]:
# IMportação das bibliotecas basicas

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

# Abaixo listamos os arquivos da base


for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
#Leitura do arquivo
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

In [ ]:
# Abaixo poemos veriricar o conteúdo da base
df

In [ ]:
#Verificamos agora os tipos de dados
df.info()

In [ ]:
# Verificando os valores nulos
df.isnull().sum()

Observamos que a base contém muitos valores nulos. Para que possamos prosseguir com o nosso trabalho de construir um modelo é necessário corrigir a base preenchendo as informações faltantes.

Abaixoi a definição dos valores a serem inseridos para cada variável:

MORTDUE-  Montante devido na hipoteca existente - Assumido como 0
VALUE - Valor da propriedade atual -  Assumido como 0
REASON DebtCon = consolidação da dívida HomeImp = melhoria da casa - 'Other' (Outro)
JOB Categorias profissionais JOBSix = 'None' (Nenhum)
YOJ Anos no emprego atual - Assumido 0
DEROG Número de principais relatórios depreciativos - Assumido como 0
DELINQ Número de linhas de crédito inadimplentes - Assumido como 0
CLAGE Idade da linha comercial mais antiga em meses - Assumido como 0
NINQ Número de linhas de crédito recentes - Assumido como 0
CLNO Número de linhas de crédito - Assumido como 0
DEBTINC Razão Dívida / Renda - Assumido como 0


In [ ]:
#Imputação de dados nas colunas
df.loc[df['MORTDUE'].isnull(),'MORTDUE'] = 0
df.loc[df['VALUE'].isnull(),'VALUE'] = 0
df.loc[df['JOB'].isnull(),'JOB'] = 'None'
df.loc[df['REASON'].isnull(),'REASON'] = 'Other'
df.loc[df['YOJ'].isnull(),'YOJ'] = 0
df.loc[df['DEROG'].isnull(),'DEROG'] = 0
df.loc[df['DELINQ'].isnull(),'DELINQ'] = 0
df.loc[df['CLAGE'].isnull(),'CLAGE'] = 0
df.loc[df['NINQ'].isnull(),'NINQ'] = 0
df.loc[df['CLNO'].isnull(),'CLNO'] = 0
df.loc[df['DEBTINC'].isnull(),'DEBTINC'] = 0

In [ ]:
#Listagem das classes de JOB
df['JOB'].unique()

In [ ]:
#Listagem das classes de REASON
df['REASON'].unique()

In [ ]:
#Definição da função para transformar a informação textual da coluna REASON em codificacao numerica
def REASONN (row):
   if row['JOB'] == 'Other':
      return 0
   if row['JOB'] == 'HomeImp':
      return 1
   if row['JOB'] == 'DebtCon':
      return 2
   return 3
df['REASONN'] = df.apply (lambda row: REASONN(row), axis=1)

In [ ]:
#Definição da função para transformar informar textual da coluna JOB em codificacao numerica
def JOBN (row):
   if row['JOB'] == 'Other':
      return 0
   if row['JOB'] == 'Office':
      return 1
   if row['JOB'] == 'Sales':
      return 2
   if row['JOB'] == 'Mgr':
      return 3
   if row['JOB'] == 'ProfExe':
      return 4
   if row['JOB'] == 'Self':
      return 5
   return 6
df['JOBN'] = df.apply (lambda row: JOBN(row), axis=1)

In [ ]:
#Contagem de valores nulos na base
df.isnull().sum()

Podemos veriricar que a base se encontra livre de espaços vazios. 
Vamos agora excluir as colunas que não serão utilzadas em nosso modelo.
A variavel resposta - BAD - também será removida porque iremos separar as variáveis de entrada, ou seja, as variaveis independentes.
Utilizaremos as variáveis numericas e excluiremos as variaveis com innformação textual.

In [ ]:
# Separando as colunas para a construção do modelo
feats = [c for c in df.columns if c not in ['BAD','REASON','JOB']]

In [ ]:
#Conteudo da base de dados
df.T

In [ ]:
#Dividindo a base em treino e teste
from sklearn.model_selection import train_test_split

train,test = train_test_split(df, test_size=0.20, random_state=42)

train.shape,valid.shape,test.shape

In [ ]:
# Instanciando o random forest classifier
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
#Aplicando o RandomForest à base
rf.fit(train[feats], train['BAD'])

In [ ]:
# Prevendo o BAD de teste usando o modelo treinado
y_test_pred = rf.predict(test[feats]).astype(int)

In [ ]:
#Importação do pacote para aferição d acuracia
from sklearn.metrics import accuracy_score

In [ ]:
# Medida da acurácia
accuracy_score(test['BAD'], y_test_pred)

Pudemos observar que o nosso modelo comporta-se muito bem pois obteve uma medida de acurácia elevada.
Para corroborar os nossos resultdos é conveniente utiliar também uma outra medida, a da área sob a curva ROC.

In [ ]:
#Vamos usar agora um método de medida mais confiável - AUROC - Area sob a curva ROC
#Geramos as probabilidades das classes na previsão (necessário para a rotina de medida AUROC)
y_test_prob = rf.predict_proba(test[feats])

In [ ]:
#Pega so uma coluna para efetuar o teste
y_test_prob = [p[1] for p in y_test_prob]

In [ ]:
#Importando o pacote para a medida da acuracia AUROC
from sklearn.metrics import roc_auc_score

In [ ]:
#Medida da acurácia  usando a area sob a curva ROC - AUROC
roc_auc_score(test['BAD'], y_test_prob) 

A medida de acurácia calculada com base na área sob a curva ROC obteve um resultado melhor ainda.
Abaixo podemos verificar graficamente o formato da curva ROC que explica esse valor de acurácia elevada.

In [ ]:
#Importação de paaotes para plotagem de graficos
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
#Curva ROC para os dados originais


fpr, tpr, threshold = metrics.roc_curve(test['BAD'], y_test_prob)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)

#plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
    
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Para completar a nossa análise, construiremos agora a nossa matriz confusão, ela também permite aferir a qualidade da resposta do modelo.

In [ ]:
#Prevendo os dados para a matriz de confusão
cnf_matrix = metrics.confusion_matrix(test['BAD'], y_test_pred)
cnf_matrix

In [ ]:
# import required modules
# is scikit's classifier.predict() using 0.5 by default?

#In probabilistic classifiers, yes. It's the only sensible threshold from a mathematical viewpoint, as others have explained.
import seaborn as sns 

%matplotlib inline
fig, ax = plt.subplots()
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Matriz Confusão', y=1.1)
plt.ylabel('Real')
plt.xlabel('Predito')

In [ ]:
#Por ultimo geramos o relatorio com medidas da qualidade das predições
classific = metrics.classification_report(test['BAD'], y_test_pred)

No relatório abaixo temos mais algumas informações sobre o nosso modelo.

O relatório mostra as principais métricas de classificação com base nas classes de resposta. Ele dá uma percepção mais global sobre a acurácia do modelo e dexa mais claro se ele tem uma performance menor para classes minoritárias.

Abaixo descrevemos as medidas do relatório:

Precision - é a habilidade do modelo em não prever um resultado positivo se o valor é realmente negativo. Em cada classe ele é definido como a razão entre os poditivos verdadeiros e a soma dos positivos verdadeiros e falsos.

Recall - é a habilidade do classificador encontrar todas as instancias positivas. Em cada classe é definido como a razão entre os postivos verddeiros e asoma dos verdadeiros positivos e falsos negativos.

F1 score - é a medida harmonica de precision e recall tal que o melhor score é 1.0 e o pior 0.0. Normalmente, F1 score é menor que a acuracia medida pois combina precision e recall. É util para comparar modelos, não como medida global de acuracia.

Support - número de ocorrencias da classe na base. Desbalanceamento no treinamento da bse pode levar a fraquezas no modelo não reveladas pelo score de acuracia, o que pode justificar uma amostragem estratificada, igualando a proporção de classes  e rebalanceamento. O suporte não muda com o modelo mas dá um diagnostico do processo de avaliação do processo.

In [ ]:
print(classific)

Novamente os nossos resultados se mostraram satisfatórios. A medida "support" indica um desbalanceamento.
Abaixo efetuaremos uma ramostrgem utilziando a rotina SMOTE, que efetua uma interpolação para inserir novas linha d eforma a igualar as classes.

In [ ]:
#Importação da biblioteca para reamostragem
from imblearn.over_sampling import SMOTE, ADASYN   #reamostragem com a rotina SMOTE

In [ ]:
X_resampled, y_resampled = SMOTE().fit_resample(train[feats], train['BAD'])

In [ ]:
y_resampled.value_counts()

In [ ]:

rf2 = RandomForestClassifier(n_jobs=-1, n_estimators=200, oob_score=True, random_state=42)

In [ ]:
#Aplicando o RandomForest à base
rf2.fit(X_resampled, y_resampled)

In [ ]:
# Prevendo o BAD para a base com reamostragem SMOTE
y_test_pred_2 = rf2.predict(test[feats]).astype(int)

In [ ]:
# Medida da acurácia
accuracy_score(test['BAD'], y_test_pred_2)

Podemos observar uma melhora na medida de acuracia.

In [ ]:
#Vamos usar agora um método de medida mais confiável - AUROC - Area sob a curva ROC
#Geramos as probabilidades das classes na previsão (necessário para a rotina de medida AUROC)
y_test_prob_2 = rf2.predict_proba(test[feats])

In [ ]:
#Pega so uma coluna para efetuar o teste
y_test_prob_2 = [p[1] for p in y_test_prob_2]

In [ ]:
#Medida da acurácia  usando a area sob a curva ROC - AUROC
roc_auc_score(test['BAD'], y_test_prob_2) 

A medida baseada na area da curva ROC também melhorou.

In [ ]:
#Curva ROC para os dados reamostrados


fpr, tpr, threshold = metrics.roc_curve(test['BAD'], y_test_prob_2)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)

#plt.plot(np.linspace(0,1,10), np.linspace(0,1,10), label="diagonal")
    
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

Ao comparar os grafico percebemos algumas pequenas diferencas. A linha da curva se afasta mais cedo do eixo vertical nesse ultimo gráfico, porém a curva se aproxima mais rapidamente do limite superior.
AS área da curva é ligeiramente superior.

In [ ]:
#Prevendo os dados para a matriz de confusão
cnf_matrix = metrics.confusion_matrix(test['BAD'], y_test_pred_2)
cnf_matrix

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Matriz Confusão', y=1.1)
plt.ylabel('Real')
plt.xlabel('Predito')

Percebemos que a reamostragem não efetuou grandes mudanças no modelo mas podemos perceber algumas diferenças.
A quantidade de negativos verdadeiros, celula em azul, teve uma ligeira queda, mas compesnada pelo aumento de verdadeiros positivos.

In [ ]:
#Geramos o relatorio com medidas da qualidade das predições
classific = metrics.classification_report(test['BAD'], y_test_pred_2)
print(classific)

# Conclusão



O modelo construido utilizando RandomForest mostrou-se bastate eficaz para o caso. De fato conseguimos obter uma grande acurácia, seja pelo método comum, seja pelo método da área da curva ROC, indicado para classes desbalanceadas.
Percebemos que, ainda que pequena, a reamostragem efetuada utilzando o método SMOTE, que insere linhas com dados caculados por interpolação, provocou algumas mudanças na performance do modelo.
A acurácia pelo método comum que verifica a razão de acertos teve uma ligeira melhora. Entretanto, a área sob a curva ROC foi ligeiramente menor. Ao observar os resultados do relatório de classificao, percebemos que caso se queira um maior rigor na concessão de crédito, o segundo modelo se mostra mais adequado, pois a medida "precision" é ligeiramente superior, 0.86 contra 0.82, para o caso de clientes com historico ruim quanto ao crédito, o que indica que ele melhor prevê esses casos. O lado negativo é que a previsibilidade de bons clientes piora, 0.92 contra 0.94 do modelo sem reamostragem, o que pode levar a perda de oportunidde de oferecer credito a alguns clientes bons pagadores.
Por fim, concluimos que os resultados obtidos com o nosso modelo viabilizam a sua utilização como medida para disponibilização de credito ao cliente.